In [1]:
import os
import numpy as np
import pyfastx
import matplotlib.pyplot as plt
import pandas as pd

#https://biopython.org/docs/1.75/api/Bio.SeqIO.QualityIO.html
#http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec79
# https://biopython.org/wiki/Split_large_file
# https://pypi.org/project/pyfastx/
# https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html


#examples:
# 69kB training_data/Sau/ugly/MS/180727-18-5425-18-01680_S8_L001_R1_001.fastq.gz
# 52MB training_data/Sau/ugly/MS/200204_20-00746_19-03927_S9_L000_R2_001.fastq.gz
# 250MB training_data/Sau/ugly/MS/200327_20-04029_20-00557_S26_L000_R2_001.fastq.gz
# 1.5GB training_data/Sau/ugly/MS/181002-18-6992-776-18_S2_L001_R2_001.fastq.gz

#### convert ascii quality with offset of 33 to phred score

In [2]:
def phred_score(quality):
    return ord(quality)-33

#### extract quality means and base contents per position

In [3]:
def extract_fastq_positions(path, max_len_estim=1000):
    phred_sum = max_len_estim*[0] # sum all qualities per position, initialize with enough space for max_len
    counts = max_len_estim*[0] # count occurences of each position
    base_content = {'G': max_len_estim*[0], #count occurences of each base per position
                    'C': max_len_estim*[0],
                    'A': max_len_estim*[0], 
                    'T': max_len_estim*[0],
                    'N': max_len_estim*[0]}
    # fill lists with values from fastq file
    for name, seq, qual in pyfastx.Fastq(path, build_index=False):
        for i in range(len(seq)):
            phred_sum[i] += phred_score(qual[i])
            counts[i] += 1
            base_content[seq[i]][i] += 1
    
    phred_sum = np.array(phred_sum)
    counts = np.array(counts)
    max_len = np.min(np.where(counts==0)) #find index of first 0 value as maximum length of sequences
    phred_sum = phred_sum[0:max_len]
    counts = counts[0:max_len]
    means = phred_sum/counts
    
    # convert the lists in base_content to np.arrays to perform calculations
    for key in base_content:
        base_content[key] = np.array(base_content[key])
    max_len = np.min(np.where(base_content['G']==0)) #find index of first 0 value as maximum length of sequences
    for key in base_content:
        base_content[key] = base_content[key][0:max_len] #remove unused positions
    #sum all occurences to calculate percentages
    sums = (base_content['G'] + base_content['C'] + base_content['T'] +
            base_content['A'] + base_content['N'])
    ncontent_total = base_content['N'].sum()/sums.sum()
    for key in base_content:
        base_content[key] = base_content[key]/sums
    
    return means, base_content, ncontent_total

#### import fastq files from directories and extract positional data

In [4]:
#from datetime import datetime #remove

def import_reads(rootdir):
    filenames = []
    all_phred_means = []
    all_base_content = []
    all_n_content = []
    for root, dirs, files in os.walk(rootdir):
        for name in files:
            filepath = root + os.sep + name
            if filepath.endswith(".fastq.gz"):
                phred_means, base_content, n_content = extract_fastq_positions(filepath)
                all_phred_means.append(phred_means)
                all_base_content.append(base_content)
                all_n_content.append(n_content)
                filenames.append(name.replace(".fastq.gz", ""))
                #print("finished " + filepath + " " + str(datetime.now(tz=None)))
    
    fastq_positions = pd.DataFrame(all_base_content, columns=['G', 'C', 'A', 'T', 'N'])
    fastq_positions['filename']= filenames
    fastq_positions['phred_means'] = all_phred_means
    fastq_positions['n_content'] = all_n_content
    
    return fastq_positions

In [5]:
# runtime ~4hrs for 184 files
position_data = import_reads('training_data')

0 finished training_data/Sau/ugly/MS/200327_20-04028_20-00328_S25_L000_R1_001.fastq.gz 2022-03-18 17:56:34.735101
1 finished training_data/Sau/ugly/MS/200709_20-07968_20-00891_S21_L000_R1_001.fastq.gz 2022-03-18 17:57:03.841202
2 finished training_data/Sau/ugly/MS/181002-18-6992-776-18_S2_L001_R2_001.fastq.gz 2022-03-18 18:05:30.328279
3 finished training_data/Sau/ugly/MS/200327_20-04029_20-00557_S26_L000_R1_001.fastq.gz 2022-03-18 18:07:11.067387
4 finished training_data/Sau/ugly/MS/200204_20-00744_19-03870_S7_L000_R2_001.fastq.gz 2022-03-18 18:07:47.029821
5 finished training_data/Sau/ugly/MS/200709_20-07969_20-00894_S22_L000_R1_001.fastq.gz 2022-03-18 18:08:15.246733
6 finished training_data/Sau/ugly/MS/200204_20-00746_19-03927_S9_L000_R2_001.fastq.gz 2022-03-18 18:08:31.547322
7 finished training_data/Sau/ugly/MS/200204_20-00744_19-03870_S7_L000_R1_001.fastq.gz 2022-03-18 18:09:07.842854
8 finished training_data/Sau/ugly/MS/200709_20-07969_20-00894_S22_L000_R2_001.fastq.gz 2022-03-

72 finished training_data/Sau/good/HS/191113_19-10416_19-02942_S175_L000_R1_001.fastq.gz 2022-03-18 20:03:57.755252
73 finished training_data/Sau/good/HS/191113_19-10417_19-02943-1_S176_L000_R2_001.fastq.gz 2022-03-18 20:05:25.676859
74 finished training_data/Sau/good/HS/191113_19-10419_19-02944-1_S178_L000_R1_001.fastq.gz 2022-03-18 20:06:30.697855
75 finished training_data/Sau/good/HS/191113_19-10422_19-02946_S181_L000_R1_001.fastq.gz 2022-03-18 20:07:34.560688
76 finished training_data/Sau/good/HS/191113_19-10422_19-02946_S181_L000_R2_001.fastq.gz 2022-03-18 20:08:38.787032
77 finished training_data/Sau/good/HS/191113_19-10417_19-02943-1_S176_L000_R1_001.fastq.gz 2022-03-18 20:10:06.720761
78 finished training_data/Ecoli/ugly/MS/190917-19-08343-NRZ-52602_S15_L001_R1_001.fastq.gz 2022-03-18 20:10:06.792300
79 finished training_data/Ecoli/ugly/MS/190917-19-08343-NRZ-52602_S15_L001_R2_001.fastq.gz 2022-03-18 20:10:06.863227
80 finished training_data/Ecoli/ugly/MS/190917-19-08319-NRZ-51

143 finished training_data/Efcm/good/MS/190318-19-01844-UW19607_S6_L001_R1_001.fastq.gz 2022-03-18 21:08:26.291369
144 finished training_data/Efcm/good/MS/190315-19-01867-UW19729_S2_L001_R2_001.fastq.gz 2022-03-18 21:09:17.809503
145 finished training_data/Efcm/good/MS/190318-19-01846-UW19617_S8_L001_R2_001.fastq.gz 2022-03-18 21:10:03.536090
146 finished training_data/Efcm/good/NX/200117_20-00173_UW20644_S52_L000_R1_001.fastq.gz 2022-03-18 21:12:05.278187
147 finished training_data/Efcm/good/NX/200117_20-00170_UW20641_S49_L000_R1_001.fastq.gz 2022-03-18 21:13:48.440135
148 finished training_data/Efcm/good/NX/200117_20-00169_UW20640_S48_L000_R1_001.fastq.gz 2022-03-18 21:14:44.279198
149 finished training_data/Efcm/good/NX/200211_20-00763_20495_S43_L000_R1_001.fastq.gz 2022-03-18 21:16:41.103695
150 finished training_data/Efcm/good/NX/200211_20-00762_20488_S42_L000_R1_001.fastq.gz 2022-03-18 21:18:25.579169
151 finished training_data/Efcm/good/NX/200117_20-00172_UW20643_S51_L000_R1_001

In [7]:
position_data

,G,C,A,T,N,filename,phred_means,n_content
0,"[0.32093878113336727, 0.1170094850835372, 0.13...","[0.2183526138251772, 0.18767429675954145, 0.18...","[0.2933941528133793, 0.23693452704000267, 0.32...","[0.16731445222807617, 0.4583816911169187, 0.35...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",200327_20-04028_20-00328_S25_L000_R1_001,"[33.417620776173734, 33.50150813663196, 33.544...",0.000017
1,"[0.33090924015330225, 0.11568807178078015, 0.1...","[0.2043215438105494, 0.18460762719648835, 0.17...","[0.3002722463154891, 0.24100346122200095, 0.32...","[0.16448328900128795, 0.4586949766352857, 0.36...","[1.3680719371312313e-05, 5.8631654448481346e-0...",200709_20-07968_20-00891_S21_L000_R1_001,"[33.828365603544476, 33.81414156417528, 33.845...",0.000056
2,"[0.45911884643111733, 0.21853582677224745, 0.2...","[0.2583751794091662, 0.30450556484785485, 0.32...","[0.16634035136121336, 0.15651535816212606, 0.1...","[0.11040827098255464, 0.3198799016645852, 0.22...","[0.005757351815948446, 0.0005633485531864895, ...",181002-18-6992-776-18_S2_L001_R2_001,"[32.69561897188973, 32.744396810585656, 32.805...",0.000200
3,"[0.3244660302401879, 0.12056002198727275, 0.13...","[0.22245728914609045, 0.18703068031773915, 0.1...","[0.29544436777942, 0.244664626821782, 0.331847...","[0.15763087136100065, 0.4477432293999051, 0.35...","[1.4414733010315182e-06, 1.4414733010315182e-0...",200327_20-04029_20-00557_S26_L000_R1_001,"[33.49598261391002, 33.542478296217, 33.580710...",0.000076
4,"[0.32172026693126327, 0.11348275258273507, 0.1...","[0.2087569797077764, 0.1843343255705364, 0.183...","[0.2940601957236522, 0.23672444989202124, 0.32...","[0.17010642230393588, 0.4654526352652775, 0.36...","[0.0053561353333722445, 5.8366894297554425e-06...",200204_20-00744_19-03870_S7_L000_R2_001,"[32.88341796533007, 33.40426661997315, 33.5569...",0.000939
...,...,...,...,...,...,...,...,...
179,"[0.3385563176826655, 0.13238338065542932, 0.14...","[0.21369912508978034, 0.2030454365262217, 0.22...","[0.2645027230318909, 0.2208120676464608, 0.281...","[0.18244167387103521, 0.4437591151718882, 0.34...","[0.0008001603246281156, 0.0, 0.0, 0.0, 0.0, 0....",191113_19-10152_20476_S134_L000_R1_001,"[34.24606319657467, 34.32009338617024, 34.3357...",0.000094
180,"[0.34024654613986394, 0.13022352898770873, 0.1...","[0.21521294727681858, 0.20165857376044655, 0.2...","[0.2643713021940959, 0.2216304588497603, 0.283...","[0.17929280186196916, 0.44648743840208444, 0.3...","[0.0008764025272524111, 0.0, 0.0, 0.0, 0.0, 0....",191113_19-10154_20478_S136_L000_R1_001,"[34.25091218982199, 34.3256000164776, 34.33550...",0.000086
181,"[0.34419330386180713, 0.1311335338191556, 0.14...","[0.2169814911974366, 0.20518074343055848, 0.22...","[0.2598899534755933, 0.23166125730098858, 0.28...","[0.17805226423744494, 0.4320223965944995, 0.34...","[0.00088298722771802, 2.0688547978397845e-06, ...",191113_19-10149_20473_S131_L000_R1_001,"[34.23408843774981, 34.338555674537695, 34.334...",0.000091
182,"[0.34140152314845806, 0.12929563395004195, 0.1...","[0.21593384242261962, 0.20109988002245074, 0.2...","[0.2641401008223354, 0.21999402686879194, 0.28...","[0.17851577987981648, 0.4496104591587154, 0.34...","[8.75372677044124e-06, 0.0, 0.0, 0.0, 0.0, 0.0...",191113_19-10154_20478_S136_L000_R2_001,"[33.93793710704778, 34.00655293687533, 34.0548...",0.002128


#### export dataset

In [6]:
position_data.to_json('exported_datasets/from_fastq_raw.json')